#### Plot of expected depreciation rates according to UIP approximation
This corresponds to the spread in net yields between countries

In [73]:
# Load data
import plotly.express as px
import utils
from importlib import reload
reload(utils)
from utils import *

spread_data.head()

There are in total
0 series which do not have
the 6 digit unit multiplier.
Yields are present for the following countries: 
GB, AU, CA, CH, CL, DE, ES, I9, FR, IL, JP, KR, MX, NZ, SE, US, ZA
Only considering these ones in the analysis (data on other countries will be dropped.)


,Country,OBS_VALUE_yield,REF_AREA,US_yield,yield_spread
TIME_PERIOD,,,,,
2004-07-01,Great Britain,5.10830,GB,4.5,0.60830
2004-07-01,Spain,4.28150,ES,4.5,-0.21850
2004-07-01,Chile,6.43180,CL,4.5,1.93180
2004-07-01,Eurozone,4.33957,I9,4.5,-0.16043
2004-07-01,France,4.27000,FR,4.5,-0.23000


In [37]:
# Plot of expected depreciation rates, starting July 2004
plot_data = spread_data[['REF_AREA','yield_spread']].copy()
plot_data = plot_data[~(plot_data['REF_AREA']).isin(
    ['DE','SE', 'FR', 'ES'] # countries to exclude
)]
plot_data.sort_values(by='REF_AREA',inplace=True)
plot_data.sort_index(inplace=True)

px.line(
    plot_data,
    y='yield_spread',
    line_group='REF_AREA',
    color='REF_AREA',
    labels={
        'TIME_PERIOD' : 'Time',
        'yield_spread' : 'Percentage point difference',
        'REF_AREA' : 'Country',
    },
    title='Spread btw domestic (+) and US (-) yields',
)

#### Plots of Nominal and real ERs vs trade as % of gdp

In [ ]:
from importlib import reload
reload(utils)
from utils import *

